In [1]:
import pandas as pd

In [2]:
print("hello")

hello


In [3]:
!pip3 install torch 'transformers[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 866.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 863.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 5.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.9 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.9 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.1 MB/s eta 0:00:0000:0100:01m


In [3]:
# test if installation is successful
import transformers
from transformers import pipeline
print(pipeline('sentiment-analysis')('we love you'))

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998704195022583}]


In [5]:
#pipelines, the highest-level entity of the transformers framework
MODEL_NAME = 'gpt2'
pipe = pipeline(task='text-generation', model=MODEL_NAME, device='cpu')

config.json: 100%|██████████| 665/665 [00:00<00:00, 3.50MB/s]
model.safetensors: 100%|██████████| 548M/548M [00:01<00:00, 275MB/s] 
generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 779kB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 1.38MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 175MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 1.45MB/s]


In [6]:
print(pipe('The elf queen'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The elf queen was almost dead – and if we continue, a dragon queen could kill us forever." A voice in the distance replied:\n\n"I\'m too afraid now to worry," Aylas sighed. With that, he entered his bedroom'}]


In [ ]:
#in transformers, “pipeline” is very different from “model”. “Model” is the thing we download from the Hub, gpt2 in our case, which is, in fact, a valid PyTorch model with some additional restrictions and naming conventions introduced by the transformers framework. “Pipeline” is the object which runs the model under the hood to perform a certain high-level task, e.g. text-generation. The correspondence is not one-to-one, you can use various models for text-generation: gpt2, gtp2-medium, gpt2-large, fine-tuned GPT-2 versions, and custom user models. But you cannot use models with no generation capabilities, such as Bert, in this pipeline

#While pipelines are what Hugging Face newbies typically start with, for us, they are not very interesting. Pipelines perform a lot of steps under the hood, which are hard to understand and even harder to reproduce. They are hard to customize and totally useless for model training or fine-tuning, custom models, performing custom tasks, or in general, everything the developers in Hugging Face did not plan in advance. You only really know the transformers framework if you can do things in a pipeline-free way.


In [6]:
#text generation example without pipelines
model = transformers.GPT2LMHeadModel.from_pretrained(MODEL_NAME)
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)


In [7]:
#Neural networks are not able to work with raw text; they only understand numbers. We need a tokenizer to convert a text string into a list of numbers. But first, it breaks the string up into individual tokens, which most often means “words”, although some models can use word parts or even individual characters.
#Tokenization is a classical natural language processing task. Once the text is broken into tokens, each token is replaced by an integer number called encoding from a fixed dictionary. Note that a tokenizer, and especially its dictionary, is model-dependent:
#you cannot use Bert tokenizer with GPT-2, at least not unless you train the model from scratch. Some models, especially of the Bert family, like to use special tokens, such as [PAD], [CLS], [SEP], etc. GPT-2, in contrast, uses them very sparingly.
enc = tokenizer(['The elf queen'], return_tensors='pt')
#return_tensors=’pt’ option means returning PyTorch tensors; lists are returned otherwise
print('enc =', enc)
print(tokenizer.batch_decode(enc['input_ids']))
#The batch_decode() method decodes tokens back to the string “The elf queen”.


enc = {'input_ids': tensor([[  464, 23878, 16599]]), 'attention_mask': tensor([[1, 1, 1]])}
['The elf queen']


In [8]:
out = model.generate(input_ids=enc['input_ids'],
attention_mask=enc['attention_mask'], max_length=20)
#Generate the text using the generate() method of our model
print('out=', out)
print(tokenizer.batch_decode(out))
#then decode the new tokens.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


out= tensor([[  464, 23878, 16599,    11,   508,   550,   587,  4953,   329,   262,
         10325,   286,   262, 31959,    11,   550,   587,  4953,   329,   262]])
['The elf queen, who had been waiting for the arrival of the elves, had been waiting for the']


In [9]:
#But there is a problem in the code, how many number of times you run it result is same.
#Because the model that we have gotten without using pipelines is not having that randomness by default. But the pipelines tweaks the config to have randomness.

config = transformers.GPT2Config.from_pretrained(MODEL_NAME)
print(config)

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size": 50257
}



In [10]:
config.do_sample = \
          config.task_specific_params['text-generation']['do_sample']

config.max_length = \
          config.task_specific_params['text-generation']['max_length']

#The dict task_specific_params contains parameter adjustments for pipeline tasks, in this case text-generation. To activate these parameters, we copy them by hand to the object


In [11]:
out = model.generate(input_ids=enc['input_ids'],
attention_mask=enc['attention_mask'], max_length=20)
#Generate the text using the generate() method of our model
print('out=', out)
print(tokenizer.batch_decode(out))
#then decode the new tokens.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


out= tensor([[  464, 23878, 16599,    11,   508,   550,   587,  4953,   329,   262,
         10325,   286,   262, 31959,    11,   550,   587,  4953,   329,   262]])
['The elf queen, who had been waiting for the arrival of the elves, had been waiting for the']


In [12]:
# Still we dont have radomness because we have to create the model again with config as the parameter

model = transformers.GPT2LMHeadModel.from_pretrained(MODEL_NAME,config=config)
out = model.generate(input_ids=enc['input_ids'],
attention_mask=enc['attention_mask'], max_length=20)
#Generate the text using the generate() method of our model
print('out=', out)
print(tokenizer.batch_decode(out))



/home/codespace/.python/current/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


out= tensor([[  464, 23878, 16599,   373,  6189,   407,  4609,    11,   673,   655,
          2227,   326,   290,   262,  1109,   673,   925,   326,   373,   262]])
['The elf queen was obviously not interested, she just wanted that and the fact she made that was the']


In [15]:
#we want to use GPT2LMHeadModel, the model with a generation head.

#In order to understand the generation, let’s try to generate a text without using the generate() method. If we run the model inference:

out = model(input_ids=enc['input_ids'], attention_mask=enc['attention_mask'])
out

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[-36.2871, -35.0111, -38.0791,  ..., -40.5161, -41.3757, -34.9191],
         [-79.0400, -78.2945, -82.4191,  ..., -83.7919, -79.5649, -80.6095],
         [-90.1009, -89.9107, -96.2588,  ..., -98.8548, -91.0607, -92.7675]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-0.9420,  1.9023,  0.8722,  ..., -1.2703, -0.4792,  1.2469],
          [-1.1995,  2.8433,  1.5851,  ..., -2.1430, -1.9376,  2.2601],
          [-1.0027,  3.2033,  1.7138,  ..., -2.5717, -0.6664,  1.7964]],

         [[ 0.1103,  0.6967, -1.1409,  ..., -0.1243,  1.8249, -0.1592],
          [ 0.1993, -2.6633,  0.5469,  ..., -1.3604,  1.6516, -0.3785],
          [ 2.0240, -2.7411, -0.0383,  ...,  0.3912,  3.5645,  0.3797]],

         [[-0.0985, -0.0323,  0.7536,  ..., -1.1902, -1.6401,  0.6545],
          [ 1.8882, -1.0223, -1.3784,  ..., -2.4035, -0.5824,  1.4703],
          [ 1.6562, -1.7104, -0.1439,  ..., -1.8056,  0.0350, -0.0496]],

    

In [16]:
import torch

In [18]:
#The code for sequence generation is the following:

input_ids = enc['input_ids']
for i in range(20):
  attention_mask = torch.ones(input_ids.shape, dtype=torch.int64)
  logits = model(input_ids=input_ids,
                attention_mask=attention_mask)['logits']
  new_id = logits[:, -1, :].argmax(dim=1) # Generate new ID
  input_ids = torch.cat([input_ids, new_id.unsqueeze(0)], dim=1)
 
print(tokenizer.batch_decode(input_ids))

['The elf queen, who had been waiting for the arrival of the elves, had been waiting for the arrival of the']


In [ ]:
# The above output is non random generation.
# to get the random generation Instead of the argmaxj (ztj), we randomly sample probability distribution pj of generating token j=1..V:
# pj = 1/Z exp( zTj / Θ) = softmax(zTj / Θ) , where Z = 𝛴k  exp( zTk / Θ),
#zTj are logits at the last position, and Θ is the temperature.


In [4]:
from transformers import AutoTokenizer

In [12]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 128kB/s]
config.json: 100%|██████████| 483/483 [00:00<00:00, 709kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 24.8MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 816kB/s]


In [34]:
inputs = tokenizer("let's try to tokenize")
print(inputs['input_ids'],inputs['attention_mask'])

[2, 408, 22, 18, 1131, 20, 20, 2853, 2952, 3] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [18]:
tokens = tokenizer.tokenize("let's try to tokenize")
tokens

['let', "'", 's', 'try', 'to', 'token', '##ize']

In [20]:
tokenizer = AutoTokenizer.from_pretrained("albert-base-v1")
print(tokenizer("let's try to tokenize"))
print(tokenizer.tokenize("let's try to tokenize"))

config.json: 100%|██████████| 684/684 [00:00<00:00, 3.46MB/s]
spiece.model: 100%|██████████| 760k/760k [00:00<00:00, 991kB/s]
tokenizer.json: 100%|██████████| 1.31M/1.31M [00:00<00:00, 6.63MB/s]


{'input_ids': [2, 408, 22, 18, 1131, 20, 20, 2853, 2952, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['▁let', "'", 's', '▁try', '▁to', '▁to', 'ken', 'ize']


In [ ]:
#observe the difference between the tokens generated,eg., ##ize and ▁let etc.,

In [26]:
#lets tokenize two sentenses.

tokenizer = AutoTokenizer.from_pretrained("albert-base-v1")
raw_inputs=[
    "This is a test for tokenzation using hugging face models",
    "This is another test sentense",
]

inputs=tokenizer(raw_inputs,padding=True,truncation=True,return_tensors='pt')
inputs

{'input_ids': tensor([[    2,    48,    25,    21,  1289,    26,    20,  2853,   380,   857,
           568, 20676,   276,  2761,     3],
        [    2,    48,    25,   226,  1289,   795,  6498,     3,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
#model

In [46]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer("let's try to tokenize",return_tensors="pt")
print(inputs)
print(inputs['input_ids'],inputs['attention_mask'])

from transformers import AutoModel
import torch

model = AutoModel.from_pretrained(model_name)
with torch.no_grad():
    logits = model(**inputs).logits
logits

# This is giving error because automodel will only execute the body without head
# if you want to classify this we have to add claasification head to auto model

{'input_ids': tensor([[  101,  2292,  1005,  1055,  3046,  2000, 19204,  4697,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[  101,  2292,  1005,  1055,  3046,  2000, 19204,  4697,   102]]) tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])


model.safetensors: 100%|██████████| 268M/268M [00:01<00:00, 220MB/s] 


AttributeError: 'BaseModelOutput' object has no attribute 'logits'

In [53]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
raw_inputs = [
    "let's try to tokenize",
    "let's try to tokenize twice",
]
inputs = tokenizer(raw_inputs,padding=True,return_tensors="pt")

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
logits = model(**inputs).logits
print(logits.shape)
print(logits)

torch.Size([2, 2])
tensor([[ 3.6117, -3.0095],
        [ 2.6642, -2.1822]], grad_fn=<AddmmBackward0>)


In [54]:
#Post Processing

predictions = torch.nn.functional.softmax(logits,dim=1)
print(predictions)
predictions.

tensor([[0.9987, 0.0013],
        [0.9922, 0.0078]], grad_fn=<SoftmaxBackward0>)